In [2]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
import numpy
from tools import *

importing Jupyter notebook from tools.ipynb


In [1]:
class CrossEntropyLoss(nn.Module):
    def __init__(self, n_class, m, s):
        
        super(CrossEntropyLoss, self).__init__()
        self.m = m
        self.s = s
        self.fc = nn.Linear(192, n_class)
        self.ce = nn.CrossEntropyLoss()

    def forward(self, x, label=None):
        
        output = self.fc(x)
        
        loss = self.ce(output, label)
        prec1 = accuracy(output.detach(), label.detach(), topk=(1,))[0]

        return loss, prec1

ModuleNotFoundError: No module named 'tools'

In [ ]:
class AMSoftmax(nn.Module):
    def __init__(self, n_class, m, s):
        super(AMSoftmax, self).__init__()
        
        self.m = m
        self.s = s
        self.weight = torch.nn.Parameter(torch.FloatTensor(n_class, 192), requires_grad=True)
        self.ce = nn.CrossEntropyLoss()
        nn.init.xavier_normal_(self.weight, gain=1)

    def forward(self, x, label=None):

        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), self.m)
        costh_m = cosine - one_hot
        costh_m_s = self.s * costh_m
        
        loss    = self.ce(costh_m_s, label)
        prec1   = accuracy(costh_m_s.detach(), label.detach(), topk=(1,))[0]
        
        return loss, prec1

In [ ]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
from tools import *

class AAMsoftmax(nn.Module):
    def __init__(self, n_class, m, s):
        
        super(AAMsoftmax, self).__init__()
        self.m = m
        self.s = s
        self.weight = torch.nn.Parameter(torch.FloatTensor(n_class, 192), requires_grad=True)
        self.ce = nn.CrossEntropyLoss()
        nn.init.xavier_normal_(self.weight, gain=1)
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.th = math.cos(math.pi - self.m)
        self.mm = math.sin(math.pi - self.m) * self.m

    def forward(self, x, label=None):
        
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.mul(cosine, cosine)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where((cosine - self.th) > 0, phi, cosine - self.mm)
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output = output * self.s
        
        loss = self.ce(output, label)
        prec1 = accuracy(output.detach(), label.detach(), topk=(1,))[0]

        return loss, prec1

In [10]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
from tools import *

class ElasticLoss_am(nn.Module):
    def __init__(self, n_class, s=64.0, m=0.05,std=0.2,plus=False):
        super(ElasticLoss_am, self).__init__()
        
        self.n_class = n_class
        self.s = s
        self.m = m
        self.weight = torch.nn.Parameter(torch.FloatTensor(n_class, 192), requires_grad=True)
        nn.init.xavier_normal_(self.weight, gain=1)
        self.std=std
        self.plus=plus
        self.ce = nn.CrossEntropyLoss()

    def forward(self, x, label=None):

        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        one_hot = torch.zeros_like(cosine)
        index = torch.where(label != -1)[0]
        margin = torch.normal(mean=self.m, std=self.std, size=label[index, None].size(), device=cosine.device).clamp(self.m-self.std, self.m+self.std)
        one_hot.scatter_(1, label.view(-1, 1), margin)
        costh_m = cosine - one_hot
        costh_m_s = self.s * costh_m
        
        loss    = self.ce(costh_m_s, label)
        prec1   = accuracy(costh_m_s.detach(), label.detach(), topk=(1,))[0]
        
        return loss, prec1

In [36]:
class ElasticLoss_aam(nn.Module):
    def __init__(self, n_class, s=64.0, m=0.05,std=0.1,plus=False):
        super(ElasticLoss_aam, self).__init__()
        #self.in_features = in_features
        self.n_class = n_class
        self.s = s
        self.m = m
        self.weight = torch.nn.Parameter(torch.FloatTensor(n_class, 192), requires_grad=True)
        nn.init.xavier_normal_(self.weight, gain=1)
        self.std=std
        self.plus=plus
        self.ce = nn.CrossEntropyLoss()
        
    def forward(self, embeddings, label):
        cos_theta = F.linear(F.normalize(embeddings), F.normalize(self.weight))
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        index = torch.where(label != -1)[0]
        m_hot = torch.zeros(index.size()[0], cos_theta.size()[1], device=cos_theta.device)
        margin = torch.normal(mean=self.m, std=self.std, size=label[index, None].size(), device=cos_theta.device).clamp(self.m-self.std, self.m+self.std) # Fast converge 
        if self.plus:
            with torch.no_grad():
                distmat = cos_theta[index, label.view(-1)].detach().clone()
                _, idicate_cosie = torch.sort(distmat, dim=0, descending=True)
                margin, _ = torch.sort(margin, dim=0, descending=True)
            m_hot.scatter_(1, label[index, None], margin[idicate_cosie])
        else:
            m_hot.scatter_(1, label[index, None], margin)
        cos_theta.acos_()
        cos_theta[index] += m_hot
        cos_theta.cos_().mul_(self.s)
        
        loss = self.ce(cos_theta, label)
        prec1 = accuracy(cos_theta.detach(), label.detach(), topk=(1,))[0]

        return loss, prec1

NameError: name 'nn' is not defined